# Using CMEMS ocean colour data

Version: 1.0

Author: Hayley Evers-King, Plymouth Marine Laboratory

Date: 07/05/2019
    
Edited on 18/09/2019 by Marie Smith (CSIR) for GMES&Africa EO course Swakopmund.
Edited on 17/05/2020 by Marie Smith (CSIR) for AOS online EO course.
Edited on 07/08/2020 by Marie Smith (CSIR) for GMES&Africa EO Training MOI.

This Jupyter notebook aims to introduce you to some of the ocean colour products available from CMEMS, and how to investigate them using some simple Python programming.

Note - this Jupyter Notebook has been tested in Python 2 and 3. You will need to ensure you have the netCDF4 and cartopy modules installed in your Python environment. 

If using an anaconda distribution use the following commands in your command line prompt

conda install -c anaconda netcdf4

conda install -c conda-forge cartopy


# Import relevant modules

The first thing to be done in any Python script, is to import any necessary modules that we need to work with the data. These are the core tools/functionality of Python and can include various groups of functions allowing use to read, manipulate, and visualise data. 

In [ ]:
import netCDF4 as nc
import numpy as np
import datetime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import warnings
warnings.filterwarnings('ignore')

# Tell Python where the data is and what it is called

After importing the relevant modules, we must tell the code where to find the data we want to work with. Below the paths are defined to two example datasets that this script will work with by default. To make this simple, the default below assumes that the data are in the same folder as this script. You can put data anywhere you like, but you will need to change the paths below. 


In [ ]:
# e.g. MYPATH = os.path.join("C:/","Users","me","Desktop")
MYPATH = "<please insert your path from Data_Path_Checker.ipynb here, removing the quotes and chevrons>"

In [ ]:
FN_CCI_L3Daily = 'dataset-oc-glo-chl-multi_cci-l3-chl_4km_daily-rep-v02_1568787454001.nc' # This command creates a variable with the name of a data file
FN_CCI_L4Monthly = 'dataset-oc-glo-chl-multi_cci-l4-chl_4km_monthly-rep-v02_1568786737268.nc' # This command creates a variable with the name of another data file 
FN_CCI_L48day = 'dataset-oc-glo-chl-multi_cci-l4-chl_4km_8days-rep-v02_1568786820923.nc'

These data sets come from the same processor - in this case the one developed through the European Space Agency Climate Change Initiative project (see www.oceancolour.org for further details). The key difference between these examples are the temporal resolutions (daily vs 8day vs monthly). They were extracted from the CMEMS catalogue for the same time and space subset as detailed below:

![title](Prod_desc2.png)

![title](Subset_reg2.png)





# Read the data 

Once we have defined where the data is and what it is called, we can read it in. The data is in NetCDF format (denoted by the .nc extension). To read this data we use the (previously imported) NetCDF4 module. 


In [ ]:
input_root = os.path.join(MYPATH,'CMEMS_test_data')

CCI_L3Daily = nc.Dataset(os.path.join(input_root, FN_CCI_L3Daily)) # This command opens the daily file
CCI_L48day = nc.Dataset(os.path.join(input_root, FN_CCI_L48day)) # This command opens the weekly (8day) file
CCI_L4Monthly = nc.Dataset(os.path.join(input_root, FN_CCI_L4Monthly)) # This command opens the monthly file

Now that we've created variables that identify and link to the netcdf files containing the data, we can use Pythons object orientation to see what we can do with this data.


In [ ]:
CCI_L3Daily #This code snippet will not run - but since we are in an interactive mode of Python, press tab, and you can see the methods associated with the object we have just created.


We can print the names of the variables contained within this obhect using the command below.

In [ ]:
print(CCI_L48day.variables)

Now we need to extract the various variables we need to work with this data. We will extract the chlorophyll a concentration, the geo-coordinates (latitude and longitude) and the fill_value that is used to mask any data we should not use (due to quality or any other type of masking)

In [ ]:
CHL_Daily =  CCI_L3Daily.variables['CHL'][:].data
CHL_Monthly = CCI_L4Monthly.variables['CHL'][:].data
CHL_8day = CCI_L48day.variables['CHL'][:].data

fillval_CD = CCI_L3Daily.variables['CHL']._FillValue
fillval_CM = CCI_L4Monthly.variables['CHL']._FillValue
fillval_CE = CCI_L48day.variables['CHL']._FillValue

Daily_lat = CCI_L3Daily.variables['latitude'][:]
Daily_lon = CCI_L3Daily.variables['longitude'][:]
day8_lat = CCI_L48day.variables['latitude'][:]
day8_lon = CCI_L48day.variables['longitude'][:]
Monthly_lat = CCI_L4Monthly.variables['latitude'][:]
Monthly_lon = CCI_L4Monthly.variables['longitude'][:]

Daily_time = CCI_L3Daily.variables['time'][:]
Monthly_time= CCI_L4Monthly.variables['time'][:]
day8_time = CCI_L48day.variables['time'][:]

# Here we convert the time stamps in to recognisable text strings, using datetime functions.
Time_D=[]
for time in np.arange(len(Daily_time)):
    daily_time_temp=(datetime.datetime.fromtimestamp(Daily_time[time]))
    Time_D.append(daily_time_temp.strftime("%d/%m/%Y"))

Time_M=[]
for time in np.arange(len(Monthly_time)):
    monthly_time_temp=(datetime.datetime.fromtimestamp(Monthly_time[time]))
    Time_M.append(monthly_time_temp.strftime("%d/%m/%Y"))
    
Time_E=[]
for time in np.arange(len(day8_time)):
    day8_time_temp=(datetime.datetime.fromtimestamp(day8_time[time]))
    Time_E.append(day8_time_temp.strftime("%d/%m/%Y"))

By typing 'whos' we can view all the variables we currently have in our interactive memory space, and some information about them

In [ ]:
whos

# Visualising the data quickly 

Let's start by having a look at the data. One of the simplest ways to do this in Python is to use the imshow function. 

The data is 3 dimensional, as we saw when we printed the variable attributes above, and we know the three indexes represent the time, latitude, and longitude dimensions respectively. In the command below, we select the first time slice of the data (note that Python uses zero indexing!), and the use the numpy squeeze funtion to compress the variable to 2 dimensions for plotting. 

In [ ]:
plt.imshow(np.squeeze(CHL_Daily[0,:,:]))
plt.show()


This is not a very good plot, I'm sure you'll agree! The colour scheme is not useful, it is hard to see the coastline, and you can't differentiate the land from cloud). The axes don't provide any information on the location (other than within the array) and there is no colour bar.

With regards to the colour scheme, it currently implies there are only two values (purple and yellow) in this array! Anyone who has worked with any data based on measurements of the ocean will know this consistency in space is impossible, and they would be correct. In this case, the default colour scheme has saturated at the higher and lower ends. In part, this is due to the fill values that are present for the invalid ocean data masks inherent in this product.  We can screen these out - this can be done in a number of ways in Python, but one of the simplest is to use the 'not a number' or NaN function.


In [ ]:
CHL_Daily[CHL_Daily == fillval_CD] = np.nan 

# While we're here - lets do the same for the monthly and the 8-day data too!

CHL_Monthly[CHL_Monthly == fillval_CM] = np.nan
CHL_8day[CHL_8day == fillval_CE] = np.nan

We can then replot the data using the same command as above

In [ ]:
plt.imshow(np.squeeze(CHL_Daily[0,:,:]))
plt.show()

The data represented by the fill value is now masked as 'NaN' values. However, we still aren't seeing any range in the chlorophyll a concentration. This is because the distribution of chlorophyll a concentration in the ocean is log distributed, and covers several orders of magnitude. We can use a log 10 function (another one from the numpy library) to account for this.

In [ ]:
plt.imshow(np.squeeze(np.log10(CHL_Daily[0,:,:])))
plt.show()

# Making high quality plots

This plot is still not very useful as we cannot tell location or the scale of the chlorophyll concentrations. This is not the sort of plot we could share with others, or publish in a paper. Below is an example code snippet to make, and save a nicer version of the plot above, using the cartopy library.

In [ ]:
fig1 = plt.figure(figsize=(7, 7), dpi=150)
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
f1 = plt.pcolormesh(Daily_lon, Daily_lat, np.squeeze(np.log10(CHL_Daily[0,:,:])), shading='flat', vmin=np.log10(0.1), vmax=np.log10(10), cmap=plt.cm.viridis)
m.coastlines(resolution='50m', color='black', linewidth=1)
m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 13, 'color': 'gray'}
g1.ylabel_style = {'size': 13, 'color': 'gray'}
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07, ticks=[ np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.1','0.5','1','3','10'], fontsize=13) 
cbar.set_label('Chlorophyll-a concentration (mg.m$^{-3}$)', fontsize=13)
plt.title('CMEMS OC-CCI Chlorophyll-a concentration, 01 January 2012 (mg.m$^{-3}$)', fontsize=13);
plt.show()

# Comparing time scales

The plot above corresponds to one day of data for the Atlantic region off the coast of Namibia. You can see plenty of influence from clouds. Let's look at a monthly composite from the same region

In [ ]:
fig1 = plt.figure(figsize=(7,7), dpi=150)
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
f1 = plt.pcolormesh(Monthly_lon, Monthly_lat, np.squeeze(np.log10(CHL_Monthly[0,:,:])), shading='flat', vmin=np.log10(0.1), vmax=np.log10(10), cmap=plt.cm.viridis)
m.coastlines(resolution='50m', color='black', linewidth=1)
m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 13, 'color': 'gray'}
g1.ylabel_style = {'size': 13, 'color': 'gray'}
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07,ticks=[np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.1','0.5','1','3','10'], fontsize=13) 
cbar.set_label('Chlorophyll-a concentration (mg.m$^{-3}$)', fontsize=13)
plt.title('CMEMS OC-CCI Chlorophyll-a concentration, January 2012 (mg.m$^{-3}$)', fontsize=13);
plt.show()

By comparing the two plots you can see the impact of the compositing process. At monthly temporal composite - you get much more data coverage than any individual daily scene would provide. However you may also lose the definition of extreme events in the averaging process. An 8day composite can sometimes provide a compromise, as it might be able to pick up some of the extreme events whilst also providing more coverage in cloudy areas - it all depends on what you are looking for...

In [ ]:
fig1 = plt.figure(figsize=(7,7), dpi=150)
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
f1 = plt.pcolormesh(day8_lon, day8_lat, np.squeeze(np.log10(CHL_8day[0,:,:])), shading='flat', vmin=np.log10(0.1), vmax=np.log10(10), cmap=plt.cm.viridis)
m.coastlines(resolution='50m', color='black', linewidth=1)
m.add_feature(cfeature.LAND, facecolor='0.75')
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 13, 'color': 'gray'}
g1.ylabel_style = {'size': 13, 'color': 'gray'}
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07,ticks=[np.log10(0.1),np.log10(0.5), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.1','0.5','1','3','10'], fontsize=13) 
cbar.set_label('Chlorophyll-a concentration (mg.m$^{-3}$)', fontsize=13)
plt.title('CMEMS OC-CCI Chlorophyll-a concentration, 01 - 08 January 2012 (mg.m$^{-3}$)', fontsize=13);
plt.show()

# Plotting Time Series 

To understand the impact of compositing the data, and to make relevant interpretations for different applications, we may wish to plot a time series of the data over a specific location. First, let's extract a spatial subset of the data - for this example we'll look at the coast off Walvisbaai.

In [ ]:
Lon_min = 13
Lon_max = 15

Lat_min = -24
Lat_max = -22

Lat_subset_d=(np.where((Daily_lat > Lat_min)&(Daily_lat<Lat_max)))
Lon_subset_d=(np.where((Daily_lon > Lon_min)&(Daily_lon<Lon_max)))
Lat_subset_m=(np.where((Monthly_lat > Lat_min)&(Monthly_lat<Lat_max)))
Lon_subset_m=(np.where((Monthly_lon > Lon_min)&(Monthly_lon<Lon_max)))

Lat_subset_d=np.squeeze(np.array(Lat_subset_d))
Lon_subset_d=np.squeeze(np.array(Lon_subset_d))

Lat_subset_m=np.squeeze(np.array(Lat_subset_m))
Lon_subset_m=np.squeeze(np.array(Lon_subset_m))

CHL_daily_subset_TS = CHL_Daily[:,Lat_subset_d[0]:Lat_subset_d[-1], Lon_subset_d[0]:Lon_subset_d[-1]]
CHL_monthly_subset_TS = CHL_Monthly[:,Lat_subset_m[0]:Lat_subset_m[-1], Lon_subset_m[0]:Lon_subset_m[-1]]


Now we've extracted our subsets, let's average over the region we've extracted, and plot in time.


In [ ]:
CHL_d_mean = np.nanmean(np.nanmean(CHL_daily_subset_TS,1),1)
CHL_m_mean = np.nanmean(np.nanmean(CHL_monthly_subset_TS,1),1)

fig1 = plt.figure(figsize=(5, 4), dpi=150)
plt.plot(Time_M, CHL_m_mean,color='k',linewidth=1.0)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Chlorophyll-a concentration (mg.m$^{-3}$)', fontsize=12)
plt.xticks(fontsize=8, rotation='vertical')
plt.yticks(fontsize=10)
plt.title('Monthly chlorophyll-a concentration for 2012', fontsize=12);
plt.show()

In the plot above we can see the seasonal cycle of chlorophyll including blooms in spring and autumn.

In [ ]:
fig1 = plt.figure(figsize=(5, 4), dpi=150)
plt.plot(Time_D, CHL_d_mean,color='k',linewidth=1.0)
plt.axhline(CHL_m_mean[0], color='r', linestyle='-')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Chlorophyll-a concentration (mg.m$^{-3}$)', fontsize=12)
plt.xticks(fontsize=8, rotation='vertical')
plt.yticks(fontsize=10)
plt.title('Daily chlorophyll-a concentration for January 2012', fontsize=12)

From the plot above (with a red line added to indicate the mean value suggested for this month from the monthly product), you can see there is a lot of variability in chlorophyll a concentration, over a monthly period. However you can also see gaps in the data.

Comparing the two plots above, you can begin to understandthe trade offs between coverage and resolution of processes (seasonal vs event scale) that must be considered when choosing which data to use.

# Other options for working with multi-temporal spatial data. 

There are many other ways this data could be sliced and/or averaged to provide information about the chlorophyll-a patterns in different regions in time. Other things you may want to explore is the use of Hovmoller plots, comparison with in situ data, calculation of indexes (Eutrophication, bloom seasonal phenology) etc.

Lets look at an example of a simple Latitude averaged Hovmoller plot for the coast off Walvisbaai using the CMEMS 8-day data of 2012

In [ ]:
Lon_min = 11
Lon_max = 15

Lat_min = -24
Lat_max = -22

Lat_subset_e=(np.where((day8_lat > Lat_min)&(day8_lat<Lat_max)))
Lon_subset_e=(np.where((day8_lon > Lon_min)&(day8_lon<Lon_max)))

Lat_subset_e=np.squeeze(np.array(Lat_subset_e))
Lon_subset_e=np.squeeze(np.array(Lon_subset_e))

CHL_8day_subset_TS = CHL_8day[:,Lat_subset_e[0]:Lat_subset_e[-1], Lon_subset_e[0]:Lon_subset_e[-1]]
# the shape that the data are stored in is [Axis_0=time, Axis_1=Lat, Axis_2=Lon]

CHL_8day_hovmoller = np.nanmean(CHL_8day_subset_TS,axis=1) #for Latitude-averaged we use Axis_1

In [ ]:
fig1 = plt.figure(figsize=(6,10), dpi=150)
f1 = plt.pcolormesh(day8_lon[Lon_subset_e[0]:Lon_subset_e[-1]], Time_E, np.squeeze(np.log10(CHL_8day_hovmoller)), shading='flat', vmin=np.log10(0.1), vmax=np.log10(10), cmap=plt.cm.viridis)
cbar = plt.colorbar(f1, orientation="horizontal", fraction=0.05, pad=0.07,ticks=[np.log10(0.1),np.log10(0.3), np.log10(1),np.log10(3),np.log10(10)]) 
cbar.ax.set_xticklabels(['0.1','0.3','1','3','10'], fontsize=13) 
cbar.set_label('Chlorophyll-a concentration (mg.m$^{-3}$)', fontsize=13)
plt.title('CMEMS OC-CCI Chlorophyll-a concentration, Latitude averaged for 2012 (mg.m$^{-3}$)', fontsize=12);
plt.show()